<a href="https://colab.research.google.com/github/vanderbilt-data-science/MNPSCollaborative/blob/Parsing-and-Schema-Bug-Fix-Section-4-and-Add-Batch-Run%2C-Adjudication%2C-and-Scoring/mnps_eval_reliability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNPS Evaluation and Reliability Testing Framework
> A notebook to help with the experimental design framework for the project.  
> DSI DSSG + MNPS  
> August 12, 2025  
> Drafted by Wayne Birch - [contact him](wayne.birch@mnps.org) for questions, code update needs, or other questions about the notebook!

This notebook builds off the starting point for the mini Hackathon with Metro Nashville Public Schools (MNPS) and the VU Data Science Institute (VU DSI). You aren't constrained to what is in this notebook, and please feel free to use your creativity to deliver the best solution

## **1** | Overview
* **Project Summary**: We’re checking whether the AI Assistant assigns the **right MNPS job title** (and major/minor role) when it reads a job description. We’ll use **a random mix of MNPS internal and external descriptions** from [New Sample_08.07.2025.csv](https://github.com/vanderbilt-data-science/MNPSCollaborative/blob/main/New%20Sample_08.07.2025.csv). For each record, the Assistant outputs a title and roles; then **a human evaluator** reviews the prediction and marks it right or wrong.

  Our go/no-go rule is simple: the model **passes only** if—even after accounting for normal sampling wiggle room—its **true accuracy is at least 90%**. We measure that with a conservative 95% statistical check. We’ll also look at performance separately for **internal vs. external** descriptions and across **major role groups** (e.g., Specialist, Analyst, Director). We only exclude items with an empty/too-short Position Summary; other missing fields are okay because the model (and human reviewer) can infer what’s needed.

  If we find recurring miss-patterns (like confusing seniority or over-weighting job titles vs. duties), we’ll **tune the prompt** and rerun. The notebook produces a clean adjudication sheet for the human reviewer, calculates accuracy with confidence intervals, and prints a clear **PASS/REJECT**decision.

* **Method Details**: Design. Prospective evaluation of an AI Assistant that classifies job descriptions into an MNPS top-1 job title (primary endpoint) and major/minor role (secondary endpoints).

  **Dataset.** We evaluate on a **random sample** of both internal MNPS and external job descriptions from [New Sample_08.07.2025.csv](https://github.com/vanderbilt-data-science/MNPSCollaborative/blob/main/New%20Sample_08.07.2025.csv). The **only** a priori exclusion is records lacking sufficient text in the **Position Summary** field, which is required for adjudication. Missing or sparse content in other fields (e.g., Education, Work Experience, Licenses/Certifications, KSAs) is not exclusionary because those signals can often be inferred from the Position Summary or recovered by the AI model during classification.

  **Model and Outputs.** For each description, the Assistant returns a structured JSON with: predicted title, major role, minor role, a 0–1 confidence score, a brief rationale, and a prompt version tag. JSON is schema-checked before scoring.

    **Ground truth.** A human evaluator reviews each prediction. Where used for formal reporting, we recommend dual independent review with adjudication and reporting **inter-rater reliability** (e.g., Cohen’s κ≥0.75), but the protocol supports single-evaluator adjudication for prompt-tuning cycles.

  **Primary outcome and acceptance criterion.** Top-1 title accuracy with a two-sided Clopper–Pearson 95% confidence interval. We accept the model if the lower bound ≥ 0.90. This rule is pre-specified and applied once per evaluation run.

  **Secondary outcomes.** (i) Title accuracy by Source (Internal vs. External) and by Major role group, each with Wilson 95% intervals for readability; (ii) Major/minor correctness rates; (iii) Error taxonomy counts (e.g., “seniority misread,” “wrong job family,” “duties overweighted/underweighted”).

  **Analysis plan.** The notebook calculates overall accuracy and confidence intervals, prints a PASS/REJECT decision, and exports subgroup tables and error buckets for prompt iteration. An optional checkpoint table reports the minimum number correct required for the acceptance lower-bound at common sample sizes. Prompt changes are versioned; re-tests are run on the full set after targeted fixes informed by the error taxonomy.

  **Bias & limitations.** External job descriptions vary in style and detail; misclassification risk rises when licensure or scope signals are missing. To mitigate, the prompt explicitly weights Essential Functions, Education/Experience, and Licenses/Certifications over job title wording and brand terms. Results generalize to descriptions similar in content and detail to the sample.

  **Reproducibility.** The notebook fixes the analysis rule (exact 95% CI lower-bound ≥0.90), logs (record_id, prompt_version, model_json, timestamp), exports the human adjudication sheet and scored results, and supports re-runs with updated prompts.

## **1.1** | One-Glance Notebook Pipeline
0 → 1 → 2.1 → 3 → 4 → 5 → 6 → (Human) → 7 → 8 → 9 → 10 → 11

0. Overview: Why and how (plain-English + methods).

1. Config & acceptance rule: Set knobs and the PASS rule (exact 95% LB ≥ 0.90).

2. Configuration and Acceptance Rule

  2.1 Eval Sample Load: Load New Sample_08.07.2025.csv, exclude only thin Position Summaries, assign Record_ID.

3. The Data: Supporting resources & how to run the model (keep as-is).

4. The Prompts: Dev prompt + Pydantic schema + parsing; (keep as-is, with tiny fixes).

5. Batch run: Run model for each record in df_eval → model_outputs.csv.

6. Adjudication sheet: Merge source text + predictions → MNPS_Adjudication_Sheet.csv.

  **(Human Step):** Reviewer fills Y/N + final labels → MNPS_Adjudication_Filled.csv.

7. Score & decision: Read filled sheet; compute accuracy + exact CI; PASS/REJECT; Wilson by subgroups; error buckets.

8. Subgroups & diagnostics: Breakouts (Internal/External, Major), top error categories (prompt fixes).

9. Helper (min successes): Quick table of “min correct to reach LB≥0.90” for common n.

10. Exports: Save subgroup tables and error buckets for reporting.

11. Next actions: If REJECT, patch prompt (targeted), bump version, re-run (first failures, then full).

## **1.2** | Notebook Section Breakdown and Continuity
**0. Overview (Markdown)**
  
  * Purpose: Narrative that frames the evaluation; what “PASS” means.
  * Inputs: None.
  * Outputs: Documentation only.
  * Who: You (authoring).

**1. Config and Acceptance rule (Code)**

  * Purpose: Central place for all constants (e.g., ACCEPT_LB=0.90, ALPHA=0.05, file paths).
  * Inputs: None (you set values).
  * Outputs: Globals used downstream (ACCEPT_LB, MIN_SUMMARY_CHARS, filenames).
  * Who: You (tuning).

**2. Evaluation Sample Load (Code)**
    
  * Purpose: Create a clean evaluation table.
  *Inputs: New Sample_08.07.2025.csv.
  *Logic: Only exclude rows with empty/too-short Position Summary; keep others even if sparse.
  *Outputs: df_eval with a stable Record_ID.
  *Who: Code (automated).

**3. The Data (Keep as-is)**

  * Purpose: Any resources/instructions needed to run the actual model (KSACs, roles, etc.).
  * Inputs: Reference files you use in prompts/logic.
  * Outputs: Context/material for Section 4.
  * Who: You & readers (instructions), model (as needed).

**4. The Prompts (Keep as-is; add tiny fixes if needed)**

  * Purpose: Developer prompt, Pydantic schema, OpenAI call, structured parse.
  * Inputs: One record’s text (from df_eval) per call; your prompt.
  * Outputs: Structured prediction(s) (title/major/minor/confidence/rationale), parsable by code.
  * Who: Model + your code.

  * Tiny fixes (where to place):
  • Add from typing import List, Optional right under your Pydantic import.
  • Replace any dict(*parsed.job_classification_table) with [item.model_dump() for item in parsed.job_classification_table].

**5. Batch model run (Code)**

  * Purpose: Loop over all df_eval rows, call Section-4 model code, collect outputs.
  * Inputs: df_eval, prompt/schema from Section 4.
  * Outputs: results_df (in-memory) and model_outputs.csv on disk.
  * Who: Code (automated).

**6. Build adjudication sheet (Code)**

  * Purpose: Create the file the human will grade.
  * Inputs: df_eval (source text) + results_df (predictions).
  * Outputs: MNPS_Adjudication_Sheet.csv with blank Y/N and final label fields.
  * Who: Code produces; Human fills.

**(Human) Adjudication**

  * Purpose: Gold standard decision per row.
  * Inputs: MNPS_Adjudication_Sheet.csv.
  * Outputs: MNPS_Adjudication_Filled.csv (same columns, Y/N + final labels completed).
  * Who: Human reviewer(s).

**7. Ingest Adjudication and Score (Code)**

  * Purpose: Apply the pre-specified acceptance rule and show diagnostics
  * Inputs: MNPS_Adjudication_Filled.csv.
  * Calculations:
      **Primary:** top-1 title accuracy with exact two-sided 95% CI; PASS if LB ≥ 0.90.
      **Secondary:** Wilson CIs by Source and (if present) Major; error categories.
  * Outputs: Console decision + summary; in-memory tables (by_src, by_major, err_counts).
  * Who: Code (automated).

**8. Subgroups and Diagnostics (Code)**

  * Purpose: Breakouts and error taxonomy for targeted prompt fixes.
  * Inputs: Adjudicated table from 7.
  * Outputs: Summaries for reporting/iteration (and later exported in 10).
  * Who: Code (automated), You (interpretation).

**9. Helper: min k for LB≥target (Code)**

  * Purpose: Quick reference of “how many correct do we need at size n?”
  * Inputs: Target LB, alpha.
  * Outputs: Printed thresholds for common n (100, 200, 300, …).
  * Who: Code (automated).

**10. Export Tables (Code)**

  * Purpose: Persist the subgroup and error summaries.
  * Inputs: Tables from 7–8.
  * Outputs: results_by_source.csv, results_by_major.csv (if present), results_error_buckets.csv.
  * Who: Code (automated).

**11. Next Actions (Markdown)**

  * Purpose: The iteration plan if REJECT (or what to lock if PASS).
  * Inputs: Diagnostics from 7–8.
  * Outputs: To-dos: patch prompt, bump PROMPT_VERSION, re-run fails → full set, re-score.
  * Who: You (decision + edits).



## **1.3** | Run Order Checklist
1. Run Sections 1 → 2.1 to load & gate the sample.
2. (Optional) glance at 3 for resources.
3. Confirm 4 runs cleanly on one row (schema + parse).
4. Run 5 to batch predictions → model_outputs.csv.
5. Run 6 to create MNPS_Adjudication_Sheet.csv; send to reviewer.
6. After review, place MNPS_Adjudication_Filled.csv next to the notebook.
7. Run 7 (decision) + 8–10 (diagnostics & exports).
8. If REJECT, follow 11 (patch prompt, bump version, re-run).



## **1.4** | Artifact Map (what files get created)
* model_outputs.csv → Section 5 (raw model predictions).
* MNPS_Adjudication_Sheet.csv → Section 6 (for the human).
* MNPS_Adjudication_Filled.csv → Human returns this (input to Section 7).
* results_by_source.csv, results_by_major.csv, results_error_buckets.csv → Section 10 (reporting).

## **2** | Configuration and Acceptance Rule
Again, you're completely free to just download this notebook, create a local virtual environment and get to coding in your favorite IDE. We provide this code just as a rapid method to get started, and focus our efforts on implementation through Google Colab.



In [ ]:
!pip install openai

In [ ]:
import os
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import pandas as pd
from google.colab import userdata

# set OpenAI API key environment variable using Google Colab
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
# --- CONFIG ---
ACCEPT_LB = 0.90   # lower bound target
ALPHA = 0.05       # 95% confidence
PRIMARY_FIELD = "title"  # primary metric: top-1 title accuracy

# Data location: use ONE of the following
DATA_URL = "https://raw.githubusercontent.com/vanderbilt-data-science/MNPSCollaborative/main/New%20Sample_08.07.2025.csv"
DATA_PATH = None   # e.g., "./New Sample_08.07.2025.csv" if local

# Required input columns (from the CSV headers provided)
REQUIRED_COLS = [
    "Job Description Name",
    "Position Summary",
    "Education",
    "Work Experience",
    "Essential Functions",
    "Licenses and Certifications",
    "Knowledge, Skills and Abilities",
    "Source"   # Internal / External
]

# Allowed labels for model outputs (extend if needed)
ALLOWED_MAJORS = [
    "Specialist", "Analyst", "Director", "Manager", "Technician", "Coordinator"
]

# Output filenames
MODEL_OUT_CSV = "model_outputs.csv"
ADJ_SHEET_CSV = "MNPS_Adjudication_Sheet.csv"  # to be filled by human
ADJ_FILLED_CSV = "MNPS_Adjudication_Filled.csv"  # human‑completed file name (you will upload/point to it later)

## **2.1** | Evaluation Sample Load and Only-Exclusion Rule
This section is the s the “load and gatekeeper” for the evaluation set.

* It loads the sample CSV (New Sample_08.07.2025.csv) from the repo (or a local path if you switch DATA_PATH).
* Enforces the only exclusion rule: drops rows whose Position Summary is empty/too short (≤ MIN_SUMMARY_CHARS).
* Normalizes (trims) the Position Summary text.
* Creates a stable ID (Record_ID) if the file doesn’t already have one.
* Hands off a clean table called df_eval for the later “run model → adjudicate → score” steps.

In [ ]:
# --- EVALUATION SAMPLE ---
import pandas as pd, numpy as np

DATA_URL = "https://raw.githubusercontent.com/vanderbilt-data-science/MNPSCollaborative/main/New%20Sample_08.07.2025.csv"
MIN_SUMMARY_CHARS = 10  # only a priori exclusion

df_eval = pd.read_csv(DATA_URL)
if "Position Summary" not in df_eval.columns:
    raise ValueError("Missing required 'Position Summary' column.")

# Only exclusion: thin/empty Position Summary
df_eval["Position Summary"] = df_eval["Position Summary"].astype(str).str.strip()
df_eval = df_eval[df_eval["Position Summary"].str.len() > MIN_SUMMARY_CHARS].copy()

# Stable ID for joining later
if "Record_ID" not in df_eval.columns:
    df_eval.insert(0, "Record_ID", np.arange(1, len(df_eval)+1))

print(f"Evaluation rows loaded: n={len(df_eval)}")
df_eval.head(3)

## **3** | The Data

The current prompt is a two-step prompt that is successful through the ChatGPT interface. It requires two types of data:
* The data to be classified
* Supporting resources

We need to read all of this in. Let's grab it and use it. The first thing you'll do is just straight up download a zip file of all of this information.

You can download all of the reference files from the link provided, then upload in the sidebar. You'll then unzip the directory using the code below.

Click on the folder icon in the left sidebar (kinda looks like this 🗂️) and you'll see all the files there. We'll read them in.


In [ ]:
!unzip /content/2025u-mnps-minihackathon.zip

In [ ]:
resources_dir_prefix = '/content/2025u-mnps-minihackathon/prompt-resources/'
roles_lookup = pd.read_csv(resources_dir_prefix+"MNPS Roles.csv")
determinants = pd.read_csv(resources_dir_prefix+"Competency Extended Descriptions.csv", encoding='latin1')
ksac_table = pd.read_csv(resources_dir_prefix+"MNPS KSACs.csv")
korn_ferry = pd.read_csv(resources_dir_prefix+"Korn_Ferry Lominger 38 Competencies.csv", encoding='latin1')

## **4** | The Prompts

What we have here is a direct prompt to get the response that we're looking for. We'll make this happen directly using the OpenAI Chat Completions API. Note that you can use other APIs as you like.

In [ ]:
zero_shot_prompt = \
""" Objective: Evaluate and group jobs from the "Job Description Export Specialists.xlsx" file based on similarities in job functions, not job titles.

Process:

- Compare all jobs against each other using the attributes listed in the file: Education, Work Experience, Licenses/Certifications, Essential Functions, Knowledge, Skills, Abilities, and Position Summary.
- Compare each job with reference sources using the same attributes. I have attached the reference sources for you.
- Group jobs based on similarities into:
  - Major role groupings (e.g., Specialist, Analyst, Manager)
  - Minor sub-groupings (e.g., Specialist I, II, III, IV) - not to exceed level IV
- Use the MNPS Roles and MNPS KSACs documents to help you determine major role groupings.
- Use the remaining documents to help you clarify subtle differences in role groupings and sub-groupings.
- Use a more qualitative, holistic assessment focused on functional alignment with KSACs rather than a quantitative scoring approach with defined complexity metrics

Output Format:

- Create a table with the following columns:
  - Original Job Title
  - New Job Title
  - Major Role Group
  - Minor Sub-Group
  - Justification for Grouping

- Provide an accompanying narrative explaining the rationale behind the groupings and any notable patterns or insights discovered during the analysis.

Job Title Convention:

- Follow the format: "[Function] [Role] [Level]" (e.g., "Collections Specialist II", "Accounts Payable Specialist III")

Additional Guidelines:

- Ensure all sources used are cited properly.
- Focus on the nature of the work performed rather than just the job titles.
- Consider the complexity of tasks, level of responsibility, and required competencies when determining groupings.
- Provide clear explanations for why each job was classified as it was, referencing specific job attributes and external benchmarks.

CONSTRAINTS:
1) Prioritize signals in this order: Essential Functions > Education/Experience > Licensure/Certifications. Ignore employer branding and title fluff unless supported by duties/scope.
2) Restrict major_role_group to one of: Specialist, Analyst, Director, Manager, Technician, Coordinator.
3) Always return confidence_0to1 as a conservative float in [0, 1].
4) Echo back the record_id provided in the input.

"""

Instead of asking for a table output, we will use **structured outputs**. Though this is a common approach for the outputs of LLMs/AI systems, you can learn more about this on [OpenAI's structured output documentation](https://platform.openai.com/docs/guides/structured-outputs?api-mode=responses). Note that you can find this information on almost all LLM/AI platform or package providers.

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional  # ← add this line

class JobClassification(BaseModel):
    """Represents the classification of a single job."""
    record_id: int = Field(..., description="Record_ID from the evaluation CSV.")
    job_title_original: str = Field(..., description="Original job title from input.")
    new_job_title: str = Field(..., description="Proposed MNPS-style title.")
    major_role_group: str = Field(..., description="One of: Specialist, Analyst, Director, Manager, Technician, Coordinator.")
    minor_sub_group: str = Field(..., description="Level (e.g., I, II, III, IV) or blank if not applicable.")
    grouping_justification: str = Field(..., description="One-sentence rationale anchored on Essential Functions, Education/Experience, and Licensure.")
    confidence_0to1: float = Field(..., ge=0.0, le=1.0, description="Self-rated confidence, conservative calibration.")

class JobClassificationTable(BaseModel):
    job_classification_table: List[JobClassification] = Field(..., description="One entry per input job.")
    narrative_rationale: str = Field(..., description="Optional narrative across the set.")


Create classifications using OpenAI. Of note here is:
* The **developer** prompt - this is the "system prompt" or "custom instructions" for the model. This determines the overall behavior of the model.
* The **user** prompt - this is what we send to the model like when we're chatting with ChatGPT.

In [ ]:
# Create openAI client
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Create messages to send
messages = [
    {"role": "developer", "content": zero_shot_prompt},
    {"role": "user", "content": "Classify the following job description: [Paste Job Description Here]"} # Replace with actual job description
]

# Assuming JobClassification and zero_shot_prompt are defined in the preceding code
response = client.beta.chat.completions.parse(
    model="gpt-4o", # Or another available model
    messages=messages,
    temperature=0.2, #changed from 1
    max_tokens=1000,
    response_format=JobClassificationTable,   # ← this
)

print(response.model_dump_json(indent=2))

In [ ]:
#look at response
response.choices[0].message.parsed

We can make this into a table using pandas!

In [ ]:
# Inspect the structured result (optional)
parsed = response.choices[0].message.parsed

# Extract the table (list of JobClassification objects)
table = parsed.job_classification_table

# If you expect exactly one classification, single-row dict:
first_item = table[0].model_dump()     # .model_dump() for Pydantic v2
# first_item -> {'job_title_original': ..., 'new_job_title': ..., ...}

# Or, if you want a DataFrame of all rows:
rows = [item.model_dump() for item in table]
results_df = pd.DataFrame(rows)
results_df.head()

## **5** | Batch Model Run on the Evaluation Sample

Runs the AI once per row in df_eval (your filtered New Sample_08.07.2025.csv), using your Section-4 prompt/schema.
* **Inputs:** df_eval, your zero_shot_prompt, model name, and PROMPT_VERSION.
* **Outputs:** A tidy predictions table with one row per record:

    * Record_ID, Model_Prompt_Version, Model_Run_Timestamp

    * Model_Pred_Title, Model_Pred_Major, Model_Pred_Minor

    * Model_Confidence_0to1, Model_Rationale_Short
It saves this as model_outputs.csv. (No scoring yet—just model outputs.)

In [ ]:
import os, datetime as dt
from openai import OpenAI

PROMPT_VERSION = "v1"   # bump when you change the prompt
MODEL_NAME = "gpt-4o"   # or your preferred model

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def classify_one(row):
    user_payload = f"""
Record_ID: {int(row['Record_ID'])}
Original Job Title: {row.get('Job Description Name','')}
Position Summary: {row.get('Position Summary','')}
Education: {row.get('Education','')}
Work Experience: {row.get('Work Experience','')}
Licenses and Certifications: {row.get('Licenses and Certifications','')}
Knowledge, Skills and Abilities: {row.get('Knowledge, Skills and Abilities','')}
"""
    messages = [
        {"role": "developer", "content": zero_shot_prompt},
        {"role": "user", "content": "Classify the following job description. Output 1 item in job_classification_table.\n" + user_payload}
    ]
    resp = client.beta.chat.completions.parse(
        model=MODEL_NAME,
        messages=messages,
        temperature=0.2,             # tighter, more deterministic
        max_tokens=1000,
        response_format=JobClassificationTable,
    )
    parsed = resp.choices[0].message.parsed.job_classification_table[0].model_dump()
    # Normalize to our evaluation columns
    return {
        "Record_ID": parsed["record_id"],
        "Model_Prompt_Version": PROMPT_VERSION,
        "Model_Run_Timestamp": dt.datetime.utcnow().isoformat(),
        "Model_Pred_Title": parsed["new_job_title"],
        "Model_Pred_Major": parsed["major_role_group"],
        "Model_Pred_Minor": parsed["minor_sub_group"],
        "Model_Confidence_0to1": float(parsed["confidence_0to1"]),
        "Model_Rationale_Short": parsed["grouping_justification"],
    }

pred_rows = [classify_one(r) for _, r in df_eval.iterrows()]
results_df = pd.DataFrame(pred_rows)
results_df.to_csv("model_outputs.csv", index=False)
print("Saved model_outputs.csv")
results_df.head(3)


## **5.1** | Bridge Outputs → Standard Columns
This cell normalizes what the batch run produced into the columns downstream cells expect. It supports two sources:
* a parsed `response` still in memory
* an existing DataFrame named `results_df` already built in Section 4/5

In [ ]:
# 5.1 — Bridge outputs → standard columns
import pandas as pd, numpy as np, datetime as dt

# This cell normalizes whatever Section 4 (or your batch run) produced into
# the columns downstream cells expect. It supports two sources:
#   (A) a parsed `response` still in memory
#   (B) an existing DataFrame named `results_df` you already built in Section 4/5

def _to_rows_from_parsed(parsed_table):
    # parsed_table: list of Pydantic objects
    rows = []
    for item in parsed_table:
        if hasattr(item, "model_dump"):
            rows.append(item.model_dump())  # Pydantic v2
        else:
            # Fallback if you somehow have dict-like items already
            rows.append(dict(item))
    return rows

# Try (A): pull from a `response` variable left by Section 4
_rows = None
try:
    _parsed = response.choices[0].message.parsed
    _rows = _to_rows_from_parsed(_parsed.job_classification_table)
except Exception:
    pass

# Otherwise (B): use an existing DataFrame `results_df` (if you already built it)
if _rows is not None:
    _raw_df = pd.DataFrame(_rows)
elif "results_df" in globals() and isinstance(results_df, pd.DataFrame):
    _raw_df = results_df.copy()
else:
    raise RuntimeError(
        "B1 bridge: couldn't find Section-4/5 outputs.\n"
        "Ensure you either keep `response` in memory (after parse) or have a DataFrame named `results_df`."
    )

# Map Section-4/5 columns → standardized evaluation columns
_cands = {
    "Record_ID":             ["record_id"],  # if absent, we'll backfill on merge with df_eval
    "Model_Pred_Title":      ["new_job_title", "pred_title"],
    "Model_Pred_Major":      ["major_role_group", "pred_major"],
    "Model_Pred_Minor":      ["minor_sub_group", "pred_minor"],
    "Model_Rationale_Short": ["grouping_justification", "rationale"],
    "Model_Confidence_0to1": ["confidence_0to1", "confidence", "score"],
    "Job_Title_Original":    ["job_title_original", "original_title"],
}

def _pick(df, options, default=np.nan):
    for col in options:
        if col in df.columns:
            return df[col]
    return default

# If you track prompt version elsewhere, set it here
PROMPT_VERSION = globals().get("PROMPT_VERSION", "v1")

results_df = pd.DataFrame({
    "Record_ID":             _pick(_raw_df, _cands["Record_ID"]),
    "Model_Prompt_Version":  PROMPT_VERSION,
    "Model_Run_Timestamp":   dt.datetime.utcnow().isoformat(),
    "Model_Pred_Title":      _pick(_raw_df, _cands["Model_Pred_Title"]),
    "Model_Pred_Major":      _pick(_raw_df, _cands["Model_Pred_Major"]),
    "Model_Pred_Minor":      _pick(_raw_df, _cands["Model_Pred_Minor"]),
    "Model_Confidence_0to1": _pick(_raw_df, _cands["Model_Confidence_0to1"]),
    "Model_Rationale_Short": _pick(_raw_df, _cands["Model_Rationale_Short"]),
    "Job_Title_Original":    _pick(_raw_df, _cands["Job_Title_Original"]),
})

# Persist for downstream cells (optional but convenient)
results_df.to_csv("model_outputs.csv", index=False)
print("Normalized Section-4/5 outputs → results_df (and wrote model_outputs.csv). Preview:")
results_df.head(3)


## **6** | Build Adjudication Sheet for the Human

Joins the **source text** (Position Summary, etc.) with the **model outputs** so a human can mark correctness.
* **Inputs:** df_eval + results_df from Step 5.
* **Outputs:** MNPS_Adjudication_Sheet.csv with:

  * Source columns (Record_ID, Source, Job Description Name, Position Summary, Education, Work Experience, Essential Functions, Licenses, KSAs)

  * Model columns from Step 5

  * Empty columns for the reviewer to fill:
Adj_Correct_Title (Y/N), Adj_Correct_Major (Y/N), Adj_Correct_Minor (Y/N), Adj_Final_Title, Adj_Final_Major, Adj_Final_Minor, Adj_Error_Category, Adj_Notes
This is the single artifact you hand to the adjudicator.

In [ ]:
base_cols = [
    "Record_ID","Source","Job Description Name","Position Summary","Education",
    "Work Experience","Essential Functions","Licenses and Certifications",
    "Knowledge, Skills and Abilities"
]
for c in base_cols:
    if c not in df_eval.columns:
        df_eval[c] = np.nan

adj = df_eval[base_cols].merge(results_df, on="Record_ID", how="left")

# Add empty adjudication fields
for c in [
    "Adj_Correct_Title (Y/N)","Adj_Correct_Major (Y/N)","Adj_Correct_Minor (Y/N)",
    "Adj_Final_Title","Adj_Final_Major","Adj_Final_Minor",
    "Adj_Error_Category","Adj_Notes"
]:
    if c not in adj.columns:
        adj[c] = ""

adj.to_csv("MNPS_Adjudication_Sheet.csv", index=False)
print("Wrote MNPS_Adjudication_Sheet.csv — share with the reviewer.")
adj.head(2)


## **7** | Ingest Adjudication and Compute PASS/REJECT

Reads the reviewer’s completed sheet and applies the acceptance rule.
* **Inputs:** MNPS_Adjudication_Filled.csv (same columns as the sheet from Step 6, now filled in).
  * **Process:**

    * Normalizes Y/N → booleans; if Y/N missing, falls back to comparing Model_* vs Adj_Final_*.

    * Computes top-1 title accuracy and its two-sided Clopper–Pearson 95% CI.

    * Decision: **PASS** if the exact 95% lower bound ≥ 0.90, else **REJECT.**

    * **Diagnostics:**

      * **By Source** (Internal/External) with Wilson 95% CIs.

      * **Top Error Categories** (from Adj_Error_Category) to guide prompt fixes.
* **Outputs:** Console printout with n, correct_titles, accuracy, exact/Wilson CIs, and the PASS/REJECT decision; plus subgroup and error-bucket summaries. (You can export these to CSV if you like; the earlier “export” cell handles that.)


In [ ]:
import math
from scipy.stats import beta

ALPHA = 0.05
ACCEPT_LB = 0.90

adj_filled = pd.read_csv("MNPS_Adjudication_Filled.csv")

yn = {"y": True, "yes": True, "n": False, "no": False, "": np.nan}
for col in ["Adj_Correct_Title (Y/N)","Adj_Correct_Major (Y/N)","Adj_Correct_Minor (Y/N)"]:
    adj_filled[col] = adj_filled[col].astype(str).str.strip().str.lower().map(yn)

def fallback(eq_col, model_col, final_col):
    return adj_filled[eq_col].where(adj_filled[eq_col].notna(),
           (adj_filled[model_col].fillna("").astype(str).str.strip() ==
            adj_filled[final_col].fillna("").astype(str).str.strip()))

adj_filled["is_correct_title"] = fallback("Adj_Correct_Title (Y/N)","Model_Pred_Title","Adj_Final_Title")

def wilson_ci(k, n):
    if n == 0: return (np.nan, np.nan)
    z = 1.959963984540054
    p = k/n
    denom = 1 + (z**2)/n
    centre = p + (z**2)/(2*n)
    adj = z * math.sqrt( (p*(1-p) + (z**2)/(4*n)) / n )
    lb = (centre - adj)/denom
    ub = (centre + adj)/denom
    return max(0.0, lb), min(1.0, ub)

def clopper_pearson_ci(k, n, alpha=ALPHA):
    if n == 0: return (np.nan, np.nan)
    a = alpha/2
    lb = 0.0 if k == 0 else beta.ppf(a, k, n-k+1)
    ub = 1.0 if k == n else beta.ppf(1-a, k+1, n-k)
    return float(lb), float(ub)

n = len(adj_filled)
k = int(adj_filled["is_correct_title"].sum())
acc = k/n if n else np.nan
lb, ub = clopper_pearson_ci(k, n, ALPHA)
lb_w, ub_w = wilson_ci(k, n)

print({
    "n": n,
    "correct_titles": k,
    "accuracy": round(acc,4),
    "exact95_CI": (round(lb,4), round(ub,4)),
    "wilson95_CI": (round(lb_w,4), round(ub_w,4)),
    "accept_rule": f"PASS if exact95 lower bound ≥ {ACCEPT_LB:.2f}",
    "DECISION": "PASS" if lb >= ACCEPT_LB else "REJECT"
})

# Subgroups & error buckets
by_src = adj_filled.groupby("Source", dropna=False)["is_correct_title"].agg(["count","sum"])
by_src["accuracy"] = by_src["sum"] / by_src["count"]
by_src["wilson_lb"], by_src["wilson_ub"] = zip(*[
    wilson_ci(int(r["sum"]), int(r["count"])) for _, r in by_src.iterrows()
])
print("\nBy Source:\n", by_src)

err = adj_filled[adj_filled["is_correct_title"] == False]
print("\nTop Error Categories:\n", err["Adj_Error_Category"].value_counts(dropna=False).head(15))
